# Architectural Lens: Advanced Architecture Recovery Tool

This notebook implements an enhanced version of Architectural‑Lens to perform architectural recovery on the Zeeguu project. It analyzes the code in both `Data/api` (backend) and `Data/frontend` (frontend) to generate comprehensive architecture views.

## Use Cases
- **Module Dependency Visualization:** Identify core components and their interdependencies
- **Architectural Pattern Detection:** Spot potential patterns and anti‑patterns in the design
- **Impact Analysis:** Understand how changes in one module may affect the overall architecture
- **Layered Architecture Analysis:** Detect and visualize layering violations

In [20]:
import os

# Clone the Architectural‑Lens repository if not already cloned
if not os.path.exists("Architectural-Lens"):
    !git clone https://github.com/Perlten/Architectural-Lens.git
else:
    print("Repository already cloned.")

# Install required dependencies
!pip install graphviz pydot networkx matplotlib

Cloning into 'Architectural-Lens'...
hint: core.useBuiltinFSMonitor=true is deprecated;please set core.fsmonitor=true instead
hint: Disable this message with "git config set advice.useCoreFSMonitorConfig false"


Defaulting to user installation because normal site-packages is not writeable


## Initialize Architectural‑Lens Configurations

The following cell initializes the Architectural‑Lens configuration in both the backend and frontend directories. It changes to the respective project folder, runs `archlens init` (which creates an `archlens.json` file), and then returns to the current working directory.

In [21]:
import os
import subprocess

def init_archlens(project):
    # Compute the project folder path relative to the current working directory.
    # Your current working directory is C:\Programming\Architecture\ArchitecturalRecovery,
    # so the project folders are in 'Data/api' and 'Data/frontend'.
    project_folder = os.path.join('Data', project)
    abs_project_folder = os.path.abspath(project_folder)
    print(f"Initializing Architectural‑Lens in: {abs_project_folder}")
    
    # Check if the project folder exists
    if not os.path.exists(project_folder):
        print(f"Error: The folder '{abs_project_folder}' does not exist.")
        return
    
    # Save current directory
    current_dir = os.getcwd()
    
    # Change to the project folder
    os.chdir(project_folder)
    
    # Run the 'archlens init' command as a shell command.
    # This uses the executable installed in your PATH.
    result = subprocess.run("archlens init", shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Error running 'archlens init':")
        print(result.stderr)
    else:
        print(result.stdout)
    
    # Optionally, print the generated archlens.json to confirm it was created
    config_file = 'archlens.json'
    if os.path.exists(config_file):
        print(f"{project} archlens.json content:")
        with open(config_file, 'r') as f:
            print(f.read())
    else:
        print(f"archlens.json was not created in {abs_project_folder}.")
    
    # Change back to the original directory
    os.chdir(current_dir)

# Initialize configuration for backend and frontend
init_archlens('api')
init_archlens('frontend')


Initializing Architectural‑Lens in: c:\Programming\Architecture\ArchitecturalRecovery\Data\api

api archlens.json content:
{
    "$schema": "https://raw.githubusercontent.com/Perlten/Architectural-Lens/master/config.schema.json",
    "name": "api",
    "rootFolder": "",
    "github": {
        "url": "",
        "branch": "main"
    },
    "saveLocation": "./diagrams/",
    "views": {
        "completeView": {
            "packages": [
                {
                    "packagePath": "",
                    "depth": 0
                }
            ],
            "ignorePackages": []
        }
    }
}
Initializing Architectural‑Lens in: c:\Programming\Architecture\ArchitecturalRecovery\Data\frontend

frontend archlens.json content:
{
    "$schema": "https://raw.githubusercontent.com/Perlten/Architectural-Lens/master/config.schema.json",
    "name": "frontend",
    "rootFolder": "",
    "github": {
        "url": "",
        "branch": "main"
    },
    "saveLocation": "./diagrams/",


## Basic Architecture Recovery
First, we'll run a basic analysis to understand the overall structure by combining both projects.

In [ ]:
import subprocess
import sys

# Run the render command using the installed archlens command
result = subprocess.run(["archlens", "render", "--source", "Data/api", "Data/frontend", "--output", "basic_archlens_output.dot"],
                        capture_output=True, text=True)
if result.returncode != 0:
    print("Error running 'archlens render':")
    print(result.stderr)
else:
    print(result.stdout)

# Run the dot command to convert the DOT file to PNG
result_dot = subprocess.run(["dot", "-Tpng", "basic_archlens_output.dot", "-o", "basic_archlens_output.png"],
                              capture_output=True, text=True)
if result_dot.returncode != 0:
    print("Error running 'dot' command:")
    print(result_dot.stderr)
else:
    print("Diagram generated successfully!")


python: can't open file 'c:\\Programming\\Architecture\\ArchitecturalRecovery\\Architectural-Lens\\archlens.py': [Errno 2] No such file or directory
'dot' is not recognized as an internal or external command,
operable program or batch file.


## Enhanced Analysis with Custom Filtering
Let's create a more focused view by filtering out less important modules.

In [23]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import read_dot

# Load the basic DOT file into a NetworkX graph
G = read_dot('basic_archlens_output.dot')

# Filter the graph to include only nodes with at least 3 incoming or outgoing edges
significant_nodes = [node for node in G.nodes() if len(list(G.predecessors(node))) + len(list(G.successors(node))) >= 3]
G_filtered = G.subgraph(significant_nodes)

# Save the filtered graph
nx.drawing.nx_pydot.write_dot(G_filtered, 'filtered_archlens_output.dot')

# Convert the filtered DOT file to PNG
!dot -Tpng filtered_archlens_output.dot -o filtered_archlens_output.png

FileNotFoundError: [Errno 2] No such file or directory: 'basic_archlens_output.dot'

## Separate Backend and Frontend Analysis
Analyzing the backend and frontend separately can provide clearer architectural insights.

In [ ]:
# Run Architectural‑Lens on just the backend
!python Architectural-Lens/archlens.py --source ../Data/api --output api_archlens_output.dot
!dot -Tpng api_archlens_output.dot -o api_archlens_output.png

# Run Architectural‑Lens on just the frontend
!python Architectural-Lens/archlens.py --source ../Data/frontend --output frontend_archlens_output.dot
!dot -Tpng frontend_archlens_output.dot -o frontend_archlens_output.png

## Identifying Core Components
Let's extract the most central modules in both the backend and frontend.

In [ ]:
def analyze_centrality(dot_file, title):
    from networkx.drawing.nx_pydot import read_dot
    import networkx as nx
    
    # Load the graph from the DOT file
    G = read_dot(dot_file)
    
    # Calculate centrality measures
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    
    # Sort nodes by centrality
    degree_sorted = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
    betweenness_sorted = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
    
    print(f"\n{title} - Top 10 modules by Degree Centrality:")
    for node, centrality in degree_sorted:
        print(f"{node}: {centrality:.4f}")
    
    print(f"\n{title} - Top 10 modules by Betweenness Centrality:")
    for node, centrality in betweenness_sorted:
        print(f"{node}: {centrality:.4f}")
    
    return G, degree_centrality, betweenness_centrality

# Analyze backend
api_G, api_degree, api_between = analyze_centrality('api_archlens_output.dot', 'Backend (API)')

# Analyze frontend
frontend_G, frontend_degree, frontend_between = analyze_centrality('frontend_archlens_output.dot', 'Frontend')

## Visualizing Core Components
Create a subgraph of the most important components for a clearer architectural view.

In [ ]:
def visualize_core_components(G, degree_centrality, betweenness_centrality, output_file, title):
    import networkx as nx
    import matplotlib.pyplot as plt
    
    # Calculate combined centrality for each node
    combined_centrality = {}
    for node in G.nodes():
        combined_centrality[node] = degree_centrality.get(node, 0) + betweenness_centrality.get(node, 0)
    
    # Sort and take top 20% of nodes
    sorted_nodes = sorted(combined_centrality.items(), key=lambda x: x[1], reverse=True)
    top_nodes = [node for node, score in sorted_nodes[:max(1, int(len(sorted_nodes) * 0.2))]]
    
    # Create subgraph with the top nodes
    core_G = G.subgraph(top_nodes)
    
    # Save the core components subgraph
    nx.drawing.nx_pydot.write_dot(core_G, f'core_{output_file}.dot')
    !dot -Tpng core_{output_file}.dot -o core_{output_file}.png
    
    # Quick visualization using matplotlib
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(core_G)
    nx.draw_networkx_nodes(core_G, pos, node_size=700, node_color='lightblue')
    nx.draw_networkx_edges(core_G, pos, width=1.0, alpha=0.7)
    nx.draw_networkx_labels(core_G, pos, font_size=8)
    plt.title(f"Core Components of {title}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    return core_G

# Visualize core components of backend
core_api_G = visualize_core_components(api_G, api_degree, api_between, 'api_archlens_output', 'Backend (API)')

# Visualize core components of frontend
core_frontend_G = visualize_core_components(frontend_G, frontend_degree, frontend_between, 'frontend_archlens_output', 'Frontend')

## Cross-Component Communication Analysis
Let's analyze how the backend and frontend potentially communicate.

In [ ]:
import re
import os
import json

def find_api_endpoints(api_dir):
    """Find API endpoints in the backend code"""
    endpoints = []
    
    for root, dirs, files in os.walk(api_dir):
        for file in files:
            if file.endswith('.py'):
                try:
                    with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()
                    # Look for Flask routes or REST endpoints
                    route_patterns = [
                        r'@\w+\.route\(["\']([^"\']*)["\'](.*?)\)\s*def\s+(\w+)',
                        r'@api\.\w+\(["\']([^"\']*)["\'](.*?)\)\s*def\s+(\w+)'
                    ]
                    
                    for pattern in route_patterns:
                        for match in re.finditer(pattern, content, re.DOTALL):
                            endpoint = match.group(1)
                            function_name = match.group(3)
                            relative_file = os.path.relpath(os.path.join(root, file), api_dir)
                            endpoints.append((endpoint, function_name, relative_file))
                except Exception as e:
                    print(f"Error processing {file}: {e}")
    
    return endpoints

def find_api_calls(frontend_dir):
    """Find API calls in the frontend code"""
    api_calls = []
    
    for root, dirs, files in os.walk(frontend_dir):
        for file in files:
            if file.endswith(('.js', '.jsx', '.ts', '.tsx', '.py')):
                try:
                    with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()
                    # Look for common HTTP request patterns (fetch, axios, etc.)
                    patterns = [
                        r'fetch\(["\']([^"\']*)["\'](.*?)\)',
                        r'axios\.\w+\(["\']([^"\']*)["\'](.*?)\)',
                        r'\.(get|post|put|delete)\(["\']([^"\']*)["\'](.*?)\)'
                    ]
                    
                    for pattern in patterns:
                        for match in re.finditer(pattern, content, re.DOTALL):
                            if pattern == patterns[2]:  
                                endpoint = match.group(2)
                            else:
                                endpoint = match.group(1)
                            relative_file = os.path.relpath(os.path.join(root, file), frontend_dir)
                            api_calls.append((endpoint, relative_file))
                except Exception as e:
                    print(f"Error processing {file}: {e}")
    
    return api_calls

# Execute the analysis using updated paths
try:
    api_endpoints = find_api_endpoints('../Data/api')
    frontend_api_calls = find_api_calls('../Data/frontend')
    
    print(f"\nFound {len(api_endpoints)} API endpoints in the backend:")
    for i, (endpoint, function, file) in enumerate(api_endpoints[:10], 1):
        print(f"{i}. {endpoint} → {function} in {file}")
    if len(api_endpoints) > 10:
        print(f"... and {len(api_endpoints) - 10} more")
    
    print(f"\nFound {len(frontend_api_calls)} API calls in the frontend:")
    for i, (endpoint, file) in enumerate(frontend_api_calls[:10], 1):
        print(f"{i}. {endpoint} in {file}")
    if len(frontend_api_calls) > 10:
        print(f"... and {len(frontend_api_calls) - 10} more")
    
    # Save results to JSON for further analysis
    with open('api_connections.json', 'w') as f:
        json.dump({
            'backend_endpoints': api_endpoints,
            'frontend_calls': frontend_api_calls
        }, f, indent=2)
except Exception as e:
    print(f"Error analyzing API connections: {e}")
    print("This analysis may require actual code from the Zeeguu project.")

## Insights and Recommendations
Based on the architectural analysis, we can identify key modules and patterns in the Zeeguu system.

In [ ]:
# Function to count incoming and outgoing dependencies
def analyze_dependencies(G):
    dependencies = {}
    
    for node in G.nodes():
        incoming = len(list(G.predecessors(node)))
        outgoing = len(list(G.successors(node)))
        dependencies[node] = {
            'incoming': incoming,
            'outgoing': outgoing,
            'total': incoming + outgoing
        }
    
    # Find highly coupled modules (both high incoming and outgoing)
    highly_coupled = sorted(dependencies.items(), key=lambda x: x[1]['total'], reverse=True)[:5]
    
    # Find potential core service modules (high incoming, low outgoing)
    core_services = sorted(dependencies.items(), key=lambda x: x[1]['incoming'] - x[1]['outgoing'], reverse=True)[:5]
    
    # Find implementation modules (high outgoing, low incoming)
    implementation_modules = sorted(dependencies.items(), key=lambda x: x[1]['outgoing'] - x[1]['incoming'], reverse=True)[:5]
    
    print("\nTop 5 Most Coupled Modules:")
    for node, stats in highly_coupled:
        print(f"{node}: {stats['incoming']} incoming, {stats['outgoing']} outgoing, {stats['total']} total")
    
    print("\nTop 5 Potential Core Service Modules:")
    for node, stats in core_services:
        print(f"{node}: {stats['incoming']} incoming, {stats['outgoing']} outgoing")
    
    print("\nTop 5 Implementation Modules:")
    for node, stats in implementation_modules:
        print(f"{node}: {stats['incoming']} incoming, {stats['outgoing']} outgoing")
    
    return dependencies

print("\nBackend (API) Dependency Analysis:")
api_deps = analyze_dependencies(api_G)

print("\nFrontend Dependency Analysis:")
frontend_deps = analyze_dependencies(frontend_G)

## Architecture Recommendations
Based on our analysis, we can provide recommendations for improving the architecture.

In [ ]:
def generate_recommendations(api_deps, frontend_deps):
    print("Architecture Recommendations for Zeeguu:")
    
    # Identify potentially problematic modules in backend
    api_high_coupling = sorted(api_deps.items(), key=lambda x: x[1]['total'], reverse=True)[:3]
    print("\n1. Backend Refactoring Opportunities:")
    for node, stats in api_high_coupling:
        print(f"   - Consider refactoring {node} (total dependencies: {stats['total']}) into smaller, more focused modules")
    
    # Identify potentially problematic modules in frontend
    frontend_high_coupling = sorted(frontend_deps.items(), key=lambda x: x[1]['total'], reverse=True)[:3]
    print("\n2. Frontend Refactoring Opportunities:")
    for node, stats in frontend_high_coupling:
        print(f"   - Consider reviewing {node} (total dependencies: {stats['total']}) for adherence to component principles")
    
    # General architectural recommendations
    print("\n3. General Architecture Recommendations:")
    print("   - Ensure clear separation between UI components and business logic in the frontend")
    print("   - Consider implementing a more formal layered architecture in the backend")
    print("   - Establish standardized API contracts between frontend and backend")
    print("   - Document key architectural decisions and patterns")

generate_recommendations(api_deps, frontend_deps)